In [10]:
 %pip install pyyaml

  Using cached PyYAML-6.0.2-cp39-cp39-macosx_11_0_arm64.whl (172 kB)
You should consider upgrading via the '/Users/jeong/Desktop/khealth/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import yaml
import numpy as np
from openai import OpenAI
from typing import List
import faiss
import pickle
from dotenv import load_dotenv

In [12]:
# 환경 변수 설정 (.env 또는 직접 설정)
load_dotenv()

client = OpenAI()

# 2. 텍스트 파싱
with open("data.yaml", "r", encoding="utf-8") as f:
    raw_text = yaml.safe_load(f)

In [24]:
texts = []
title = raw_text["title"]
content =  raw_text["chapters"]
for chapter in content:

    texts.append({
            "title": title,
            "chapter": chapter["name"],
            "section": None,
            "text": chapter["content"]
        })
    print(f"Added chapter: {chapter['name']}")
    if "sections" in chapter:
        for section in chapter["sections"]:
            texts.append({
                "title": title,
                "chapter": chapter["name"],
                "section": section["name"],
                "text": section["content"]
            })
            print(f"Added section: {section['name']}")

Added chapter: 서론
Added chapter: 제 1부: 인지 발달의 기초 이론

Added section: 1.1 장 피아제의 인지 발달 이론
Added section: 1.2 레프 비고츠키의 사회문화적 이론
Added section: 1.3 비교 분석: 피아제와 비고츠키

Added chapter: 제 2부: 심리사회적, 정서적, 행동적 발달 이론

Added section: 2.1 정신분석 및 심리사회적 관점
Added section: 2.2 애착 이론 (보울비와 에인스워스)
Added section: 2.3 행동주의 및 사회 학습 이론
Added chapter: 제 3부: 도덕성 및 체계적 발달 이론

Added section: 3.1 로렌스 콜버그의 도덕 발달 이론
Added section: 3.2 유리 브론펜브레너의 생태학적 체계 이론
Added chapter: 제 4부: 현대적 관점, 비판 및 적용

Added section: 4.1 기초 이론에 대한 비판적 평가
Added section: 4.2 발달 과학의 현대적 진화
Added section: 4.3 실제적 적용 및 개입
Added chapter: 결론


In [31]:
docs = [text["text"] for text in texts]
metadatas = [{k: v for k, v in text.items() if k != "text"} for text in texts]

print(docs[2])

피아제의 구성주의 입문
장 피아제(Jean Piaget)는 아동 발달에 대한 이해를 혁신적으로 바꾼 중요한 인물로 평가받습니다.3 그의 핵심 주장은 아동이 지식이 채워지기를 기다리는 '빈 그릇'이 아니라, 환경과의 상호작용을 통해 스스로 세상에 대한 지식과 이해를 적극적으로 구성해 나가는 '어린 과학자'와 같다는 것입니다.1 이러한 능동적인 지식 구성 과정이 그의 구성주의 이론의 핵심입니다.8
인지 적응의 핵심 원리
피아제는 인지 발달을 이끄는 기제로 다음과 같은 핵심 원리들을 제시했습니다.
스키마 (Schemas): 스키마는 지능적 행동의 기본 단위로, 특정 상황을 이해하고 반응하는 데 사용되는 정신적 모델 또는 개념입니다.4 아동은 경험이 쌓일수록 더 많고 복잡한 스키마를 형성하게 됩니다.4
동화 (Assimilation): 기존에 가지고 있던 스키마를 사용하여 새로운 대상이나 상황을 이해하는 과정입니다.4 예를 들어, 작고 털이 있으며 네 발을 가진 동물에 대한 스키마를 가진 아이가 스컹크를 처음 보고 "고양이"라고 부르는 것이 동화의 예입니다.10
조절 (Accommodation): 새로운 정보가 기존의 스키마와 맞지 않을 때, 기존 스키마를 수정하거나 새로운 스키마를 형성하는 과정입니다.4 앞선 예에서 아이가 스컹크는 고양이와 다르다는 것을 배우고 '스컹크'라는 새로운 스키마를 만드는 것이 조절에 해당합니다.10
평형화 (Equilibration): 이는 학습 과정을 추동하는 힘입니다. 피아제는 인간이 인지 구조와 환경 사이의 균형 상태, 즉 평형을 추구한다고 보았습니다.4 아동이 기존 스키마로 세상을 설명할 수 있을 때 평형 상태에 있지만, 새로운 정보가 기존 스키마에 맞지 않으면(동화 실패) 불균형이 발생합니다. 이 불균형 상태는 아동에게 조절을 하도록 동기를 부여하여 균형을 회복하고 발달을 촉진합니다.11
피아제의 4단계 인지 발달
피아제는 인지 발달이 질적으로 다른 네 개의 단계를 거쳐 순서대로 진행된다고 보았습니다.
감각운동기 (Sensori

In [42]:
# OpenAI Embedding
def get_embeddings(texts: List[str]) -> List[List[float]]:
    response = client.embeddings.create(input=texts, model="text-embedding-3-small")
    return [e.embedding for e in response.data]

embeddings = get_embeddings(docs)

# FAISS 벡터스토어 저장
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

# 텍스트 + 메타데이터 함께 저장
chunks = [{"doc": doc, **meta} for doc, meta in zip(docs, metadatas)]

with open("faiss_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

faiss.write_index(index, "faiss_index.idx")
print("✅ 벡터스토어 저장 완료")

✅ 벡터스토어 저장 완료


In [43]:
# 5. 검색 함수
def search(query: str, k: int = 3):
    q_embedding = get_embeddings([query])[0]
    D, I = index.search(np.array([q_embedding]).astype("float32"), k)
    with open("faiss_chunks.pkl", "rb") as f:
        stored_chunks = pickle.load(f)
    return [stored_chunks[i] for i in I[0]]

In [45]:
results = search("우리 아이가 너무 울어")
for r in results:
    print(r["chapter"], r["section"])
    print(r["doc"])

제 4부: 현대적 관점, 비판 및 적용
 4.3 실제적 적용 및 개입
육아 조언 및 전략:
에릭슨의 단계에 기반: 영아기의 신뢰감 형성, 유아기의 선택권 제공, 학령전기 아동의 주도성 격려 등 각 심리사회적 위기를 통해 자녀를 지원하기 위한 실질적인 팁을 제공합니다.36
안정 애착 증진: 요구에 대한 일관되고 신속한 반응, 신체적 애정, 규칙적인 일과 수립, 개방적인 의사소통 등 안정 애착을 촉진하기 위한 구체적인 전략을 제시합니다.43
교실 및 교육 전략:
피아제 적용: 발견 학습, 구체물 조작을 통한 실제 활동, 아동의 발달 단계에 맞는 과제 제시(예: 구체적 조작기 아동에게는 구체적 자료 필요) 등을 활용합니다.7
비고츠키 적용: 비계 설정, 그룹 활동, 또래 교습, 학생들의 ZPD 내에서 활동을 구성하는 방법을 사용합니다.23
브론펜브레너 적용: 강력한 가정-학교 연결(중간체계)의 중요성을 강조하고, 학생들의 가정생활과 지역사회(외체계)의 영향을 이해하며, 포용적인 교실 문화를 조성합니다.58
증거 기반 임상 개입:
안전의 원 (Circle of Security, COS): 어린 자녀(4개월~6세)를 둔 양육자를 위한 애착 기반 개입 프로그램입니다.46 부모가 자녀의 탐색(안전 기지)과 위안(안전한 피난처)에 대한 요구를 이해하도록 돕기 위해 애착의 시각적 지도를 사용합니다.45 핵심 목표 중 하나는 부모가 자신의 애착 역사를 성찰하고 그것이 민감하게 반응하는 능력("상어 음악")을 어떻게 방해할 수 있는지 이해하도록 돕는 것입니다.46 이 프로그램은 혼란된 애착을 줄이는 데 효과적인 것으로 나타났습니다.99
애착 및 생물행동 따라잡기 (Attachment and Biobehavioral Catch-up, ABC): 역경을 경험한 영아(6~24개월)의 양육자를 위한 10회기 가정 방문 프로그램입니다.100 애착 이론과 스트레스 신경생물학에 기반합니다.101 '실시간' 코칭과 비디오 피드백을 사용하여 양육자가 양육적 돌봄을 제공하고, 즐거움으로 아이의 주